In [ ]:
# backtest_script.py
from backtesting import Backtest, Strategy
import script  # 导入 script.py 中的内容
import os

class PredictiveStrategy(Strategy):
    def init(self):
        
        filepath = os.path.join(script.current_dir, 'model/20240325_baddest_model.h5')
        self.model = script.load_model(filepath)
        components_path = os.path.join(script.current_dir, 'model/20240325_baddest_model.pkl')
        components_dict = script.joblib.load(components_path)
        self.scaler_features = components_dict['scaler_features']
        self.scaler_label = components_dict['scaler_label']

    def next(self):
        # 获取数据并进行预测
        df = self.data.df.iloc[:self.data.df.index.get_loc(self.data.index[-1]) + 1]  # 回测只使用当前可用数据
        predict_price = script.predict(df, self.scaler_features, self.scaler_label, self.model, time_step=16)
        
        current_close_price = self.data.Close[-1]
        predict_change_rate = predict_price / current_close_price - 1

        # 执行交易策略并获取信号
        state = script.execute_trading_strategy(predict_change_rate)
        
        if state['trade_signals'] == 1:
            self.buy()
        elif state['trade_signals'] == -1:
            self.sell()

# 运行回测
import yfinance as yf
data = yf.download('QQQ', start='2023-01-01', end='2023-12-31', interval='1h')

bt = Backtest(data, PredictiveStrategy, cash=50000, commission=.002)
stats = bt.run()
bt.plot()
